In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Conv3D, Conv2DTranspose, Conv3DTranspose, Flatten, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from PIL import Image
import Hyperparameters as hyp
import sys
import os

In [9]:
train_path = 'Data/Training2/'
test_path = '../input/maxcs229test/'

In [10]:
lr_opts = [0.001, 0.0001]
decay_opts = [0.0001, 0.00001]
eps_opts = [0.00001, 0.000001]
num_filters_opts = [128, 64]
kernel_size_opts = [3, 5]

In [11]:
def get_sliding_windows(frames_list, sequence_size):
    """ For data augmenting purposes.
    Parameters
    ----------
    frames_list : list
        A list of sorted frames of shape Config.FRAME_RES X Config.FRAME_RES
    sequence_size: int
        The size of the lstm sequence
    Returns
    -------
    list
        A list of clips , 20 frames each
    """
    clips = []
 
    for i in range(len(frames_list) - sequence_size):
        clip = np.zeros(shape=(sequence_size, hyp.FRAME_RES, hyp.FRAME_RES, 1))
        clip[:, :, :, 0] = frames_list[i : i + sequence_size]
        clips.append(clip)
    return np.array(clips)

def getData(path):
    data = []
    for file in os.listdir(path):
        if file[-3:] == "npy":
            all_frames = np.load(path + file, allow_pickle=True)
            data.extend(get_sliding_windows(frames_list=all_frames, sequence_size=hyp.CLIP_LEN))
            
    return np.array(data)

In [12]:
def experiment2(train_data, val_data):
    input = Input(shape=(train_data.shape[1], train_data.shape[2], train_data.shape[3], 1))
    conv = Conv3D(hyp.filters, hyp.kernel_size, activation="relu")(input)
    bottle = Conv3D(int(hyp.filters / 2), hyp.kernel_size, activation="relu")(conv)
    bottleT = Conv3DTranspose(hyp.filters, hyp.kernel_size, activation="relu")(bottle)
    convT = Conv3DTranspose(1, hyp.kernel_size, activation="relu")(bottleT)
    model = Model(inputs=input, outputs=convT)

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=hyp.lr, decay=hyp.decay, epsilon=hyp.eps), loss="mse")
    model.summary()
    es = EarlyStopping(min_delta=hyp.eps, verbose=1, patience=3)
    model.fit(train_data, train_data, validation_data=(val_data, val_data), epochs=100, callbacks=[es])

In [13]:
train_data = getData(train_path)
train_data, val_data, _,  _ = train_test_split(train_data, train_data, test_size=0.1)
print(f"Train_data shape is {train_data.shape}")
experiment2(train_data, val_data)

Train_data shape is (40583, 10, 32, 32, 1)


In [7]:
val_loss = sys.maxsize
best_lr, best_decay, best_eps, best_num_filters, best_kernel_size = 0, 0, 0, 0, 0
for lr in lr_opts:
    for decay in decay_opts:
        for eps in eps_opts:   
            for num_filters in num_filters_opts:
                for kernel_size in kernel_size_opts: 
                    input = Input(shape=(train_data.shape[1], train_data.shape[2], train_data.shape[3], 1))
                    conv = Conv3D(num_filters, kernel_size, activation="relu")(input)
                    bottle = Conv3D(int(num_filters / 2), kernel_size, activation="relu")(conv)
                    bottleT = Conv3DTranspose(num_filters, kernel_size, activation="relu")(bottle)
                    convT = Conv3DTranspose(1, kernel_size, activation="relu")(bottleT)
                    model = Model(inputs=input, outputs=convT)
                    model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr, decay=decay, epsilon=eps), loss="mse")
                    model.summary()
                    es = EarlyStopping(min_delta=eps, verbose=1, patience=3)
                    history = model.fit(train_data, train_data, validation_data=(val_data, val_data), epochs=10000, callbacks=[es])
                    val = history.history['val_loss']
                    test = history.history['loss']
                    if val[-1] < val_loss:
                        val_loss = val[-1]
                        best_lr = lr
                        best_decay = decay
                        best_eps = eps
                        best_num_filters = num_filters
                        best_kernel_size = kernel_size
                        model.save("3DConv_Auto")
                        np.save("3DConv_Loss", np.array([val, test]))
print('The optimal training hyperparams are:')
print(f"lr = {best_lr}")
print(f"decay = {best_decay}")
print(f"eps = {best_eps}")
print(f"num_filters = {best_num_filters}")
print(f"kernel_size = {best_kernel_size}")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 128, 128, 1)] 0         
_________________________________________________________________
conv3d (Conv3D)              (None, 8, 126, 126, 128)  3584      
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 6, 124, 124, 64)   221248    
_________________________________________________________________
conv3d_transpose (Conv3DTran (None, 8, 126, 126, 128)  221312    
_________________________________________________________________
conv3d_transpose_1 (Conv3DTr (None, 10, 128, 128, 1)   3457      
Total params: 449,601
Trainable params: 449,601
Non-trainable params: 0
_________________________________________________________________
Train on 2100 samples, validate on 234 samples
Epoch 1/10000
2100/2100 [==============================] - 518s 247ms/samp

KeyboardInterrupt: 